In [1]:
import os 
import re
import pickle

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score

# Utils

In [2]:
def crawl_paths(experiment_name):
    """
    Crawls the paths and gathers the configuration and results files created from the 
    """
    paths = []
    for (dirpath, dirnames, filenames) in os.walk(experiment_name):
        files = [os.path.join(dirpath, file) for file in filenames]
        paths.extend(files)
    configs = [path for path in paths if 'config.yaml' in path]
    xvals = [path for path in paths if 'xval' in path and 'results' in path]

    full_paths = {}        
    date_re = re.compile('(.*202[0-9]_[0-9]+_[0-9]+_[0-9]+/)')
    model_re = re.compile('.*logger/(.*)/')
    fold_re = re.compile('.*f([\-0-9]+)')
    for config_path in configs:
        try:
            experiment_date_path = date_re.findall(config_path)[0]
            exp_xvals = [xval for xval in xvals if experiment_date_path in xval]
            full_paths[experiment_date_path] = {
                'config': config_path,
                'xval': exp_xvals[0],
            }
        except IndexError:
            continue

    return full_paths

In [6]:
def look_demographics(path):
    paths_lstm = crawl_paths(path)

    with open('../../data/beerslaw/features/raw/rankings.pkl', 'rb') as fp:
        post_test = pickle.load(fp)
        post_test = post_test.set_index('username')
    demographics = ['gender', 'language', 'field', 'year']
    scores = {dem: {} for dem in demographics}
    
    seed = re.compile('raw_full/([0-9]+\_[0-9]+\_[0-9]+\_[0-9]+)/')
    cluster_iter = re.compile('/([0-9]+\_[0-9]+)/')
    seed_data = {}
    
    for token in paths_lstm:
        with open(paths_lstm[token]['xval'], 'rb') as fp:
            res = pickle.load(fp)
        try:
            with open(paths_lstm[token]['config'], 'rb') as fp:
                config = pickle.load(fp)
        except pickle.UnpicklingError:
            with open(paths_lstm[token]['config'], 'r') as fp:
                config = yaml.load(fp, Loader=yaml.FullLoader)
            
        path_seed = seed.findall(token)[0] + '_' + cluster_iter.findall(token)[0]
            
        idd = config['id_dictionary']

        test_indices = []
        test_indexes = []
        y_truths = []
        y_preds = []
        y_probas = []
        learner_ids = []
        genders = []
        languages = []
        fields = []
        years = []
        for fold in range(10):
            test_indix = list(res[fold]['test_indices'])
            test_index = list(res[fold]['test_index'])
            x_test = [res['x'][idx] for idx in test_index]
            longenough = [i for i in range(len(x_test)) if len(x_test[i]) > 10]
            test_index = [test_index[i] for i in longenough]
            
            y_truth = [res['y'][idx] for idx in test_index]
            y_pred = list(res[fold]['y_pred'])
            y_pred = [y_pred[idx] for idx in longenough]
            y_proba = list(res[fold]['y_proba'])
            y_proba = [y_proba[idx] for idx in longenough]
            lid = [idd['sequences'][idx]['learner_id'] for idx in test_indix]
            lid = [lid[idx] for idx in longenough]
            gender = [post_test.loc[liid]['gender'] for liid in lid]
            language = [post_test.loc[liid]['language'] for liid in lid]
            field = [post_test.loc[liid]['field'] for liid in lid]
            year = [post_test.loc[liid]['year'] for liid in lid]

            test_indices = test_indices + test_indix
            test_indexes = test_indexes + test_index
            y_truths = y_truths + y_truth
            y_preds = y_preds + y_pred
            y_probas = y_probas + y_proba
            learner_ids = learner_ids + lid
            genders = genders + gender
            languages = languages + language
            fields = fields + field
            years = years + year

        data = {
            'y_test': y_truths,
            'y_pred': y_preds,
            'y_proba': y_probas,
            'learner_ids': learner_ids,
            'gender': genders, 
            'language': languages,
            'field': fields,
            'year': years
        }
        
        seed_data[path_seed] = data
    return seed_data

In [7]:
def true_positive(y_test, y_pred):
    positive = [i for i in range(len(y_test)) if y_test[i] == 1]
    yt = np.array([y_test[i] for i in positive])
    yp = np.array([y_pred[i] for i in positive])
    return sum(yt == yp) / len(positive)

def false_positive(y_test, y_pred):
    fpositive = [i for i in range(len(y_test)) if y_test[i] == 0]
    yt = np.array([y_test[i] for i in fpositive])
    yp = np.array([y_pred[i] for i in fpositive])
    return sum(yt != yp) / len(fpositive)

def pred_positive(y_test, y_pred):
    positive = [yy for yy in y_pred if yy == 1]
    return len(positive) / len(y_test)

def pred_roc(y_test, y_pred):
    return roc_auc_score(y_test, y_pred)
    
def print_equalised_odds_summary_per_demo(seed_data, seed, demographics): 
    demographics_list = np.unique(seed_data[seed][demographics])
    
    demos = []
    tps = []
    fps = []
    pps = []
    rocs = []
    for demo in demographics_list:
        demo_indices = [i for i in range(len(seed_data[seed][demographics])) if seed_data[seed][demographics][i] == demo]
        demo_ytest = [seed_data[seed]['y_test'][i] for i in demo_indices]
        demo_ypred = [seed_data[seed]['y_pred'][i] for i in demo_indices]
        tp = true_positive(demo_ytest, demo_ypred)
        fp = false_positive(demo_ytest, demo_ypred)
        pp = pred_positive(demo_ytest, demo_ypred)
        roc = pred_roc(demo_ytest, demo_ypred)
        tps.append(tp)
        fps.append(fp)
        pps.append(pp)
        rocs.append(roc)
        demos.append(demo)
        
        print('  attribute: {} ({})'.format(demo, len(demo_indices)))
        print('      tp: {}\n      fp: {}\n      pp: {}\n      roc: {}\n'.format(tp, fp, pp, roc))
        
        
    print('  Sorted by True Positive')
    pos_indices = np.argsort(tps)
    demos = [demos[idx] for idx in pos_indices]
    sorted_tps = [tps[idx] for idx in pos_indices]
    print_tps = ['{} ({})'.format(demos[i], sorted_tps[i]) for i in range(len(demos))]
    print('     ', print_tps)
    print()
    
    print('  Sorted by False Positives')
    neg_indices = np.argsort(fps)
    demos = [demos[idx] for idx in neg_indices]
    sorted_tns = [tps[idx] for idx in neg_indices]
    print_tps = ['{} ({})'.format(demos[i], sorted_tps[i]) for i in range(len(demos))]
    print('     ', print_tps)
    print()
    
    print('  Sorted by Predicted Positive')
    pps_indices = np.argsort(pps)
    demos = [demos[idx] for idx in pps_indices]
    sorted_pps = [pps[idx] for idx in pps_indices]
    print_pps = ['{} ({})'.format(demos[i], sorted_pps[i]) for i in range(len(demos))]
    print('     ', print_pps)
    print()
    
def print_equalised_odds_summary(seed_data, seed):
    for demographics in ['gender', 'language', 'field', 'year']:
        print('='*100)
        print('ATTRIBUTE: {}'.format(demographics))
        print_equalised_odds_summary_per_demo(seed_data, seed, demographics)
        print()
        print()
        
def average_across_seeds(seed_data):
    seeds = list(seed_data.keys())
    ypreds = []
    yprobas = []
    ytests = []
    genders = []
    languages = []
    fields = []
    years = []
    lids = []
    for learner_id in seed_data[seeds[0]]['learner_ids']:
        ypred = np.sum([seed_data[seed]['y_pred'][seed_data[seed]['learner_ids'].index(learner_id)] for seed in seeds])
        ypred /= len(seeds)
        ypred = int(ypred)
        ypreds.append(ypred)
        
        yproba = np.sum([seed_data[seed]['y_proba'][seed_data[seed]['learner_ids'].index(learner_id)] for seed in seeds])
        yproba /= len(seeds)
        yprobas.append(yproba)
        
        ytests.append(seed_data[seeds[0]]['y_test'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        genders.append(seed_data[seeds[0]]['gender'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        languages.append(seed_data[seeds[0]]['language'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        fields.append(seed_data[seeds[0]]['field'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        years.append(seed_data[seeds[0]]['year'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        lids.append(learner_id)
        
    seed_data['all_data'] = {
        'y_test': ytests,
        'y_pred': ypreds,
        'y_proba': yprobas,
        'learner_ids': lids,
        'gender': genders, 
        'language': languages,
        'field': fields,
        'year': years
    }
    return seed_data
        
        



In [8]:
path = '../../data/beerslaw/algos_results/lstm'
seed_data = look_demographics(path)
seed_data = average_across_seeds(seed_data)
eo = print_equalised_odds_summary(seed_data, 'all_data')

ATTRIBUTE: gender
  attribute: 1 (135)
      tp: 0.4909090909090909
      fp: 0.1125
      pp: 0.26666666666666666
      roc: 0.6892045454545455

  attribute: 2 (108)
      tp: 0.5283018867924528
      fp: 0.09090909090909091
      pp: 0.3055555555555556
      roc: 0.7186963979416808

  attribute: 3 (4)
      tp: 0.0
      fp: 0.0
      pp: 0.0
      roc: 0.5

  attribute: 4 (7)
      tp: 0.5
      fp: 0.0
      pp: 0.14285714285714285
      roc: 0.75

  Sorted by True Positive
      ['3 (0.0)', '1 (0.4909090909090909)', '4 (0.5)', '2 (0.5283018867924528)']

  Sorted by False Positives
      ['4 (0.0)', '2 (0.4909090909090909)', '1 (0.5)', '3 (0.5283018867924528)']

  Sorted by Predicted Positive
      ['1 (0.0)', '3 (0.14285714285714285)', '4 (0.26666666666666666)', '2 (0.3055555555555556)']



ATTRIBUTE: language
  attribute: Deutsch (178)
      tp: 0.5365853658536586
      fp: 0.10416666666666667
      pp: 0.30337078651685395
      roc: 0.716209349593496

  attribute: Français (76)


In [10]:
roc_auc_score(seed_data['all_data']['y_test'], seed_data['all_data']['y_proba'])

0.5229320229320229

In [11]:
path = '../../data/beerslaw/algos_results/ts_attention/'
seed_data = look_demographics(path)
seed_data = average_across_seeds(seed_data)
eo = print_equalised_odds_summary(seed_data, 'all_data')

ATTRIBUTE: gender
  attribute: 1 (135)
      tp: 0.5272727272727272
      fp: 0.1625
      pp: 0.3111111111111111
      roc: 0.6823863636363637

  attribute: 2 (108)
      tp: 0.5283018867924528
      fp: 0.09090909090909091
      pp: 0.3055555555555556
      roc: 0.7186963979416808

  attribute: 3 (4)
      tp: 0.0
      fp: 0.0
      pp: 0.0
      roc: 0.5

  attribute: 4 (7)
      tp: 0.5
      fp: 0.0
      pp: 0.14285714285714285
      roc: 0.75

  Sorted by True Positive
      ['3 (0.0)', '4 (0.5)', '1 (0.5272727272727272)', '2 (0.5283018867924528)']

  Sorted by False Positives
      ['1 (0.0)', '2 (0.5)', '4 (0.5272727272727272)', '3 (0.5283018867924528)']

  Sorted by Predicted Positive
      ['4 (0.0)', '3 (0.14285714285714285)', '2 (0.3055555555555556)', '1 (0.3111111111111111)']



ATTRIBUTE: language
  attribute: Deutsch (178)
      tp: 0.5609756097560976
      fp: 0.11458333333333333
      pp: 0.3202247191011236
      roc: 0.7231961382113821

  attribute: Français (76)
  

In [12]:
roc_auc_score(seed_data['all_data']['y_test'], seed_data['all_data']['y_proba'])

0.5035595035595035

In [9]:
import yaml
yaml_path = '../../data/beerslaw/algos_results/ts_attention/1_15/chemlab-nonnested/simplestate_secondslstm/binconcepts/ts_attention/raw_full/2022_03_03_0/config.yaml'
with open(yaml_path, 'rb') as fp:
    yy = pickle.load(fp)


In [10]:
yy

{'paths': {'sequenced_simulations': '../data/sequenced_simulations/'},
 'experiment': {'class_name': 'binconcepts',
  'root_name': 'chemlab-nonnested/simplestate_secondslstm/binconcepts/ts_attention/raw_full/',
  'name': '2022_03_03_0',
  'old_root_name': 'blank/simplestate_secondslstm',
  'random_seed': 129,
  'n_folds': 'blank',
  'n_classes': 2,
  'class_map': '../data/experiment_keys/permutation_maps/nconcepts_binary.yaml'},
 'data': {'min_length': 0,
  'pipeline': {'sequencer': 'simplestate_secondslstm',
   'sequencer_interval': 5,
   'sequencer_dragasclick': True,
   'concatenator': {'type': 'chemconcat', 'tasks': ['2']},
   'demographic_filter': 'chemlab',
   'event_filter': 'nofilt',
   'break_filter': 'cumulseconds',
   'break_threshold': 0.6,
   'adjuster': 'full',
   'encoder': 'raw',
   'skipgram_weights': '',
   'skipgram_map': '',
   'aggregator': 'minmax',
   'encoders_aggregators_pairs': {1: ['actionspan', 'normagg'],
    2: ['1hot', 'aveagg']}},
  'adjuster': {'limit':

In [38]:
seed_data.keys()

dict_keys(['2022_03_03_7_15_30', '2022_03_03_0_15_30', '2022_03_03_9_15_30', '2022_03_03_11_15_30', '2022_03_03_8_15_30', '2022_03_03_1_15_30', '2022_03_03_6_15_30', '2022_03_03_10_15_30', '2022_03_03_12_15_30', '2022_03_03_3_15_30', '2022_03_03_4_15_30', '2022_03_03_14_15_30', '2022_03_03_13_15_30', '2022_03_03_5_15_30', '2022_03_03_2_15_30', '2022_03_03_7_45_61', '2022_03_03_0_45_61', '2022_03_03_9_45_61', '2022_03_03_11_45_61', '2022_03_03_8_45_61', '2022_03_03_1_45_61', '2022_03_03_6_45_61', '2022_03_03_10_45_61', '2022_03_03_12_45_61', '2022_03_03_15_45_61', '2022_03_03_3_45_61', '2022_03_03_4_45_61', '2022_03_03_14_45_61', '2022_03_03_13_45_61', '2022_03_03_5_45_61', '2022_03_03_2_45_61', '2022_03_03_7_30_45', '2022_03_03_0_30_45', '2022_03_03_9_30_45', '2022_03_03_11_30_45', '2022_03_03_8_30_45', '2022_03_03_1_30_45', '2022_03_03_6_30_45', '2022_03_03_10_30_45', '2022_03_03_12_30_45', '2022_03_03_3_30_45', '2022_03_03_4_30_45', '2022_03_03_14_30_45', '2022_03_03_13_30_45', '2022

In [37]:
roc_auc_score(seed_data['all_data']['y_test'], seed_data['all_data']['y_proba'])

0.5035595035595035

In [40]:
roc_auc_score(seed_data['2022_03_03_7_15_30']['y_test'], [yp[1] for yp in seed_data['2022_03_03_7_15_30']['y_proba']])

0.7484407484407485

In [6]:
seed_data.keys()

dict_keys(['60', '69', '5', '3', '75', '16', '64', '0', '96'])

In [7]:
seed_data['60'].keys()

dict_keys(['y_test', 'y_pred', 'y_proba', 'learner_ids', 'gender', 'language', 'field', 'year'])

In [69]:
seed_data['60']['learner_ids'].index('2hr6mkdc')

3